In [18]:
import pandas as pd

In [19]:
df = pd.read_csv('../../2020.csv',encoding='euc-kr')
df.drop(columns=['등록일자'],inplace=True)
df.head()

,사용일자,노선명,역명,승차총승객수,하차총승객수
0,20200101,1호선,종각,20427,16301
1,20200101,1호선,시청,12126,10516
2,20200101,우이신설선,신설동,892,828
3,20200101,우이신설선,보문,917,855
4,20200101,우이신설선,성신여대입구(돈암),2010,2363


In [20]:
df1 = df.copy()
cols = list(df.columns)[:3]
target = list(df.columns)[3:]

In [21]:
# 역명, 노선명 --> 지하철역, 호선명
df = df.rename(columns={'역명': '지하철역', '노선명':'호선명'})

In [22]:
# '사용일' 열을 datetime 형식으로 변환
df1['사용일자'] = pd.to_datetime(df1['사용일자'], format='%Y%m%d')
# 평일과 주말 구분하는 새로운 열 생성
df1['주중/주말'] = df1['사용일자'].apply(lambda x: '주말' if x.weekday() >= 5 else '주중')
# 평일 데이터만 선택
weekday_df = df1[df1['주중/주말'] == '주중']

In [23]:
week_df = weekday_df.copy()
week_df['사용일자'] = pd.to_datetime(weekday_df['사용일자']).dt.strftime('%Y%m%d').astype(int)

In [24]:
week_df

,사용일자,노선명,역명,승차총승객수,하차총승객수,주중/주말
0,20200101,1호선,종각,20427,16301,주중
1,20200101,1호선,시청,12126,10516,주중
2,20200101,우이신설선,신설동,892,828,주중
3,20200101,우이신설선,보문,917,855,주중
4,20200101,우이신설선,성신여대입구(돈암),2010,2363,주중
...,...,...,...,...,...,...
217050,20201231,경의선,신촌,541,633,주중
217051,20201231,경의선,서울역,2512,3329,주중
217052,20201231,경원선,소요산,1522,1401,주중
217053,20201231,경원선,동두천,1379,1589,주중


In [25]:
df_list = []
for i in range(1, 13):
    start_date = 20200000 + i*100
    end_date = start_date + 100
    df_temp = week_df[(week_df['사용일자'] >= start_date) & (week_df['사용일자'] < end_date)].copy()
    df_temp['사용일자'] = 202000 + i
    df_temp = df_temp.groupby(cols)[target].agg('sum').reset_index()
    df_list.append(df_temp)
df_res = pd.concat(df_list, axis=0)

df_res.tail()

,사용일자,노선명,역명,승차총승객수,하차총승객수
598,202012,중앙선,원덕,4879,4764
599,202012,중앙선,중랑,102667,98868
600,202012,중앙선,지평,698,628
601,202012,중앙선,팔당,13923,13937
602,202012,중앙선,회기,421387,404566


In [26]:
df_res.to_csv('./res.csv', index=False)

In [27]:
df = pd.read_csv('./res.csv')
df.tail()

,사용일자,노선명,역명,승차총승객수,하차총승객수
7185,202012,중앙선,원덕,4879,4764
7186,202012,중앙선,중랑,102667,98868
7187,202012,중앙선,지평,698,628
7188,202012,중앙선,팔당,13923,13937
7189,202012,중앙선,회기,421387,404566


In [28]:
# 역명, 노선명 --> 지하철역, 호선명
df = df.rename(columns={'역명': '지하철역', '노선명':'호선명'})

In [29]:
# 호선명 리스트로 추출해서 사용
lines = df.호선명.unique().tolist()
df_dict = {line: df[df['호선명'] == line].copy() for line in lines}
for line, frame in df_dict.items():
    # frame = df[df['호선명']==line].copy()
    frame['총 승차인원'] = frame.loc[:,['승차총승객수']].sum(axis=1)
    frame['총 하차인원'] = frame.loc[:,['하차총승객수']].sum(axis=1)

    frame.to_csv(f'./{line}.csv',index=False,encoding='utf-8')

In [30]:
# 저장 위치 고치기
line_info = [
    (['./1호선.csv', './경부선.csv', './경원선.csv', './경인선.csv', './장항선.csv'], '1호선'),
    (['./2호선.csv'], '2호선'),
    (['./3호선.csv', './일산선.csv'], '3호선'),
    (['./4호선.csv', './과천선.csv', './안산선.csv'], '4호선'),
    (['./5호선.csv'], '5호선'),
    (['./6호선.csv'], '6호선'),
    (['./7호선.csv'], '7호선'),
    (['./8호선.csv'], '8호선'),
    (['./9호선.csv', './9호선2~3단계.csv'], '9호선'),
    (['./수인선.csv', './분당선.csv'], '수인분당선'),
    (['./경의선.csv', './중앙선.csv'], '경의중앙선')

]

for df_list, line_name in line_info:
    df_copies = []
    for file in df_list:
        df = pd.read_csv(file)
        df_copies.append(df.copy())
    result = pd.concat(df_copies, axis=0)
    result = result.reset_index(drop=True)
    result.호선명 = line_name
    cols = list(result.columns)[:3]
    target = list(result.columns)[3:]
    res = result.groupby(cols)[target].agg('sum').reset_index()
    res.to_csv(f'lines/{line_name}.csv', index=False)